# This is the Final build for the Project

### There are 3 main stages in this Project and one auxiliary stage which is run to set up the project. 
First, the `final_project.ipynb` is run to populate the PostGresDB and do some data cleaning. 
Later this file is run.  

This file contains 3 phases:
Data Generation -> Graph Building -> Visualization


## Stage 1: Data Generation


<div class="alert alert-block alert-danger">
<b>Danger:</b> Run Stage 0 before starting this
</div>

### Input: 
* Start time `(2019-01-01 00:00:00)`
* End time `(2019-01-01 23:59:59)`
* Poolsize `(300, 420, 600)`
    
### Output: 
* CSV / Pandas DF that contains data for the next cycle
    * The dataframe will contain these fields: 
         1. id
         2. tpep_pickup_datetime
         3. tpep_dropoff_datetime
         4. passenger_count
         5. trip_distance - acquired from OSRM
         6. PULocationID
         7. DOLocationID


## Stage 2: Graph Construction

This stage is responsible to construct graphs using networX to model the relationships between passengers. 
The connected edges represent the rides that are merged. 

### Input Parameters: 
* Poolsize
* Weight calculating functions as arguments
 

## Stage 3: Visualization

This stage is responsible to gather data from stage to for Visualization

### Input Parameters: 
* TBD
* TBD